<a href="https://colab.research.google.com/github/MuhammadShavaiz/AI_learning/blob/main/spaCy_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install --upgrade spacy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 10.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [28]:
import spacy

In [29]:
nlp = spacy.load('en_core_web_md')

In [30]:
with open('/content/data.txt' , 'r') as f:
  text = f.read()

In [31]:
doc = nlp(text)
sentence1 = list(doc.sents)[0]
print(f'{sentence1}')

In recent years, the rapid advancement of technology has dramatically changed the way we live, work, and communicate.


In [32]:
#i dont know but my code is not printing correct words, like it should be grapes and the output is Grapples!!!

In [33]:
import numpy as np

my_word = 'country'
ms = nlp.vocab.vectors.most_similar(
    np.asarray([
        nlp.vocab[my_word].vector
    ]),
    n=10
)
words = [nlp.vocab.strings[w] for w in ms[0][0]]
print(words)

['country—0,467', 'nationâ\x80\x99s', 'countries-', 'continente', 'Carnations', 'pastille', 'бесплатно', 'Argents', 'Tywysogion', 'Teeters']


In [34]:
#better approach

In [35]:
from scipy import spatial
cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)

queen = nlp.vocab['country'].vector
computed_similarities = []
for word in nlp.vocab:
    # Ignore words without vectors
    if not word.has_vector:
        continue
    similarity = cosine_similarity(queen, word.vector)
    computed_similarities.append((word, similarity))

computed_similarities = sorted(computed_similarities, key=lambda item: -item[1])
print([w[0].text for w in computed_similarities[:10]])

['country', 'countries', 'world', 'citizens', 'population', 'industries', 'sector', 'especially', 'States', 'European']


In [36]:
doc1 = nlp('I like salty fries and hamburgers.')
doc2 = nlp('Fast food tastes very good.')
print(doc1.similarity(doc2))


0.691649353055761


In [37]:
doc1 = nlp('An athlete plays sports.')
doc2 = nlp('Football is good for health.')
print(doc1.similarity(doc2))


0.4853610103544024


In [38]:
doc1 = nlp('i am going to school')
doc2 = nlp('put this in freezer')
print(doc1.similarity(doc2))


0.20700801185166268


In [39]:
doc1 = nlp('I enjoy apples.')
doc2 = nlp('I enjoy burgers.')
print(doc1.similarity(doc2))


0.9722735004203199


pipelines are used to alter textual data in a sequential manner. it ensures the output of one pipe is passed to another pipe as input in the correct order. also helps in acheiving good time complexity

In [40]:
nlp = spacy.blank('en')

In [41]:
nlp.add_pipe('sentencizer')

In [42]:
nlp.analyze_pipes()

{'summary': {'sentencizer': {'assigns': ['token.is_sent_start', 'doc.sents'],
   'requires': [],
   'scores': ['sents_f', 'sents_p', 'sents_r'],
   'retokenizes': False}},
 'problems': {'sentencizer': []},
 'attrs': {'doc.sents': {'assigns': ['sentencizer'], 'requires': []},
  'token.is_sent_start': {'assigns': ['sentencizer'], 'requires': []}}}

In [43]:
nlp2 = spacy.load('en_core_web_md')

In [44]:
nlp2.analyze_pipes()

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False}},
 'problems': {'tok2vec': [],
  'tagger': [],
  'parser': [],
  'attribute_ruler': [],
  'lemmatizer': [],
  'ner': []},
 'att

Using the EntityRuler as an example, I am extending the spaCy pipeline with custom functionalities. By adding the EntityRuler, I can define specific entity patterns using a dictionary, allowing the pipeline to recognize these predefined entities in addition to those detected by the default models. This approach enhances entity recognition by integrating custom rules alongside the existing machine learning-based components in the pipeline.
Here's a list of functionalities that can be added to spaCy pipelines, beyond what's available in default models:

EntityRuler: Adds custom entity patterns for enhanced

* TextCategorizer: Performs custom text classification or sentiment analysis.
* Custom Tokenizer: Defines specialized tokenization rules.
* Custom Vectors: Integrates domain-specific word embeddings.
* Matcher: Performs flexible pattern matching.
* Custom Token Attributes: Adds extra attributes to tokens.
* Custom Training: Fine-tunes models on specific datasets.

In [45]:
nlp = spacy.load('en_core_web_md')

In [46]:
doc = nlp('I like reading about Elam in books')

In [47]:
for ent in doc.ents:
  print(ent.text, ent.label_)

Elam PERSON


In [48]:
patterns = [
    {'label':'GPE', 'pattern':"Elam"}
]

add a before attribute because if not this addtional function will appear after ner and will not produce the correct output. either the output of entity ruler should be allowed to alter the ner or put entity ruler before sending data to ner for required results. i am using before, the later option

In [50]:
ent_ruler = nlp.add_pipe('entity_ruler', before = 'ner')

In [51]:
ent_ruler.add_patterns(patterns)

In [54]:
doc2 = nlp('I like reading about Elam in books')

In [55]:
for ent in doc2.ents:
  print(ent.text, ent.label_)

Elam GPE
